# **Linear regression model from scratch**

In [ ]:
import numpy as np
import torch

In [ ]:
inputs = np.array([[73, 67, 43],
[91, 88, 64],
[87, 134, 58],
[102, 43, 37],
[69, 96, 70]], dtype='float32')

In [ ]:
targets = np.array([[56],
[81],
[119],
[22],
[103]], dtype='float32')

In [ ]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [ ]:
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


In [ ]:
w=torch.randn(2,3,requires_grad=True)
b=torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[ 0.2831,  0.6987, -0.9328],
        [-0.5606,  0.8556,  1.0564]], requires_grad=True)
tensor([1.7788, 0.3893], requires_grad=True)


In [ ]:
def model(x):
  return x @ w.t() +b

In [ ]:
preds=model(inputs)
print(preds)
print(targets)

tensor([[ 29.1456,  62.2189],
        [ 29.3249,  92.2806],
        [ 65.9278, 127.5439],
        [ 26.1840,  19.0888],
        [ 23.0895, 117.7966]], grad_fn=<AddBackward0>)
tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


**Loss Function**

In [ ]:
diff0 = preds - targets
torch.sum(diff0 * diff0) / diff0.numel()

tensor(1307.7662, grad_fn=<DivBackward0>)

In [ ]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [ ]:
loss = mse(preds, targets)
print(loss)

tensor(1307.7662, grad_fn=<DivBackward0>)


**Compute Gradients**

In [ ]:
#loss.backward()
print(w.grad)
print(b.grad)

tensor([[-3273.4275, -4189.9644, -2595.8115],
        [  589.5712,   769.9067,   482.5929]])
tensor([-41.4656,   7.5858])


**Train the model using gradient descent**

In [ ]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [ ]:
prediction = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(1307.7662, grad_fn=<DivBackward0>)


In [ ]:
for i in range(100):
    prediction = model(inputs)
    loss = mse(prediction, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [ ]:
prediction = model(inputs)
loss = mse(prediction, targets)
print(loss)

tensor(150.3206, grad_fn=<DivBackward0>)


In [ ]:
targets

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])

# **Linear regression using PyTorch built-ins**

**Define DataLoader**

In [ ]:
import numpy as np
import torch

In [ ]:
inputs1 = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [ ]:
targets1 = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [ ]:
inputs = torch.from_numpy(inputs1)
targets = torch.from_numpy(targets1)

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [ ]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  88.,  64.],
        [ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.]])
tensor([[ 81., 101.],
        [ 56.,  70.],
        [103., 119.],
        [ 22.,  37.],
        [119., 133.]])


**nn.Linear**

In [ ]:
import torch.nn as nn

In [ ]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3495, -0.4898, -0.3139],
        [ 0.2463, -0.0677,  0.4148]], requires_grad=True)
Parameter containing:
tensor([0.0925, 0.3807], requires_grad=True)


In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.3495, -0.4898, -0.3139],
         [ 0.2463, -0.0677,  0.4148]], requires_grad=True),
 Parameter containing:
 tensor([0.0925, 0.3807], requires_grad=True)]

In [ ]:
preds = model(inputs)
preds

tensor([[ -71.7348,   31.6573],
        [ -94.9036,   43.3786],
        [-114.1550,   36.7899],
        [ -68.2293,   37.9357],
        [ -93.0178,   39.9075]], grad_fn=<AddmmBackward0>)

**loss functions**

In [ ]:
import torch.nn.functional as F

In [ ]:
loss_fn = F.mse_loss

In [ ]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(16848.7090, grad_fn=<MseLossBackward0>)


**Optimizer**

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

**Train for multiple epochs**

In [ ]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 1045.6869
Epoch [20/100], Loss: 522.3015
Epoch [30/100], Loss: 453.5011
Epoch [40/100], Loss: 400.4626
Epoch [50/100], Loss: 353.8784
Epoch [60/100], Loss: 312.8349
Epoch [70/100], Loss: 276.6686
Epoch [80/100], Loss: 244.7978
Epoch [90/100], Loss: 216.7104
Epoch [100/100], Loss: 191.9551


In [ ]:
prediction = model(inputs)
prediction

tensor([[ 61.6402,  74.8073],
        [ 82.6492, 102.2987],
        [110.4932, 122.0242],
        [ 46.7344,  62.6467],
        [ 87.7394, 107.0887]], grad_fn=<AddmmBackward0>)

In [ ]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [ ]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[59.4881, 73.6545]], grad_fn=<AddmmBackward0>)